In [21]:
def create_netcdf(inyear, outyear, nmonths, pressin, lonin, latin):
    from netcdftime import utime, JulianDayFromDate,DateFromJulianDay, date2index
    from netcdftime import datetime as datetimex, date2num, num2date
    from netCDF4 import Dataset
    import time

    # create a series of dates for the netcdf file in times.
    from datetime import datetime, timedelta

    field_o = np.load('ordonez_res'+str(inyear)+'.npy')
    field_l = np.load('liang_res'+str(inyear)+'.npy')
    field_z = np.load('ziska_res'+str(inyear)+'.npy')
    nmonths1, nlevs1, ny1, nx1 = np.shape(field_o)
    
    dataset = Dataset('N96_L85_ch2br2_'+str(outyear)+'.nc', 'w', format='NETCDF4')

    # Global Attributes
    dataset.description = 'TEST NETCDF-CF COMPLIANT SCRIPT'
    dataset.history = 'Created ' + time.ctime(time.time())
    dataset.source = ''
    dataset.Conventions = 'CF-1.0'
    dataset.standard_name_vocabulary='CF-1.0'

    time  = dataset.createDimension('time', None)
    time  = dataset.createVariable( 'time', 'f8', ('time',)) 
    time.standard_name = 'time'
    time.long_name = 'time'
    time.units = 'days since 1991-09-01 00:00:00'
    time.calendar = '360_day'
    time.axis = 'T'
    
    dates = []
    times = []
    for n in range (0,12):
        # nasty fix for the fact that datetime.timedelta only works on 365 day calendar
        new_date =  date2num(datetime(inyear,1,16),units=time.units, calendar=time.calendar)+n*30
        dates.append(num2date(new_date, units=time.units, calendar=time.calendar))
        
    time[:] = date2num(dates,units=time.units,calendar=time.calendar)
    
    # Create HDF5 *format*, classic *model*

    height = dataset.createDimension('pressure', nplevs)
    height = dataset.createVariable('pressure',  np.int32,   ('pressure',))
    height.units = 'mbar'
    height.axis='Z'
    height.positive='up'
    height.standard_name = 'pressure_levels'

    lat   = dataset.createDimension('lat',  ny1)
    lat   = dataset.createVariable('lat',   np.float32, ('lat',))
    lat.standard_name = 'latitude'
    lat.units = 'degrees_north'
    lat.cartesian_axis = "Y"
    lat.axis = 'Y'
    lat.actual_range = -90.0, 90.0

    lon   = dataset.createDimension('lon', nx1)
    lon   = dataset.createVariable('lon',  np.float32, ('lon',))
    lon.standard_name = 'longitude'
    lon.units = 'degrees_east'
    lon.cartesian_axis = "X"
    lon.axis = "X"
    lat.actual_range = 0.0, 360.0

    tracer4 = dataset.createVariable('Tracer4_conc', np.float32, ('time','pressure','lat', 'lon',), fill_value=-999.)
    tracer4.units = 'kg kg-1'
    tracer4.standard_name='Ch2Br2 mass mixing ratio'

    tracer5 = dataset.createVariable('Tracer5_conc', np.float32, ('time','pressure','lat', 'lon',), fill_value=-999.)
    tracer5.units = 'kg kg-1'
    tracer5.standard_name='Ch2Br2 mass mixing ratio'

    tracer6 = dataset.createVariable('Tracer6_conc', np.float32, ('time','pressure','lat', 'lon',), fill_value=-999.)
    tracer6.units = 'kg kg-1'
    tracer6.standard_name='Ch2Br2 mass mixing ratio'

    lon[:]   = lonin[:]
    lat[:]   = latin[:]
    height[:] = pressin[:]

    tracer4[:,:,:,:] = field_o[:,:,:,:]
    tracer5[:,:,:,:] = field_l[:,:,:,:]
    tracer6[:,:,:,:] = field_z[:,:,:,:]

    print ('success')
    print 'time values (in units %s): ' % time.units+'\n',time[:]
    dataset.close()

    return outyear

In [29]:
import subprocess as sb
import numpy as np
import netCDF4

press = np.array([1000,950,900,850,700,500,400,300,250,200,150,100,70,50,30,20,10])*100
nplevs= len(press)

filename='/Users/ptg21/data/projects/TRANSCOM/N96_ancil_creation/xkawa.p.avg2000.nc'
ncfile2     = netCDF4.Dataset(filename)

lon2        = ncfile2.variables['longitude'][:]
lat2        = ncfile2.variables['latitude'][:]
pressure    = ncfile2.variables['p'][:]
times2      = ncfile2.variables['time']
inyears = [1998]
outyears=[1998]

for outyear in outyears:
    filename_to_remove =  'N96_L85_ch2br2_'+str(outyear)+'.nc'
    print ('removing '+ filename_to_remove)
    sb.Popen(['rm', '-Rf', filename_to_remove]).wait()

for inyear in inyears:
    for outyear in outyears:
        created_for = create_netcdf(inyear = inyear, outyear=outyear, nmonths = times2,\
                      pressin = press/100. , lonin = lon2, latin = lat2)
        print
        print 'netcdf created for', created_for
        print
        

removing N96_L85_ch2br2_1998.nc
success
time values (in units days since 1991-09-01 00:00:00): 
[ 2295.  2325.  2355.  2385.  2415.  2445.  2475.  2505.  2535.  2565.
  2595.  2625.]

netcdf created for 1998

